In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.vision.widgets import *

In [3]:
from fastai.collab import *
from fastai.tabular.all import *
path=untar_data(URLs.ML_100k)

In [4]:
os.listdir(path)

['ua.test',
 'u2.test',
 'u1.test',
 'u.item',
 'u3.base',
 'README',
 'allbut.pl',
 'u.occupation',
 'u.info',
 'u1.base',
 'u.data',
 'u.genre',
 'ub.base',
 'mku.sh',
 'u5.base',
 'u3.test',
 'u4.base',
 'u.user',
 'ua.base',
 'ub.test',
 'u2.base',
 'u5.test',
 'u4.test']

In [5]:
with open(path/'README') as f:
    lines=f.read()
    print(lines)

SUMMARY & USAGE LICENSE

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
	* 100,000 ratings (1-5) from 943 users on 1682 movies. 
	* Each user has rated at least 20 movies. 
        * Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set. Detailed descriptions of
the data file can be found at the end of this file.

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under th

In [6]:
ratings=pd.read_csv(path/'u.data', delimiter='\t', header=None, names=['user', 'movie', 'rating', 'timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
movies=pd.read_csv(path/'u.item', delimiter='|', encoding='latin-1', usecols=(0,1), names=('movie', 'title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [8]:
ratings=ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [9]:
dls=CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,542,My Left Foot (1989),4
1,422,Event Horizon (1997),3
2,311,"African Queen, The (1951)",4
3,595,Face/Off (1997),4
4,617,Evil Dead II (1987),1
5,158,Jurassic Park (1993),5
6,836,Chasing Amy (1997),3
7,474,Emma (1996),3
8,466,Jackie Chan's First Strike (1996),3
9,554,Scream (1996),3


In [10]:
n_users=len(dls.classes['user'])
n_movies=len(dls.classes['title'])
n_factors=5
user_factors=torch.randn(n_users, n_factors)
movie_factors=torch.randn(n_movies, n_factors)
user_factors.shape

torch.Size([944, 5])

In [11]:
one_hot_3=one_hot(3, n_users).float()
user_factors.t() @ one_hot_3

tensor([-0.4586, -0.9915, -0.4052, -0.3621, -0.5908])

In [12]:
class Example:
    def __init__(self, a): self.a=a
    def say(self, x):return f'Hello {self.a}, {x}'
#coming from Pytorch Module
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0, 5.5)):
        self.user_factors=Embedding(n_users, n_factors)
        self.movie_factors=Embedding(n_movies, n_factors)
        self.y_range=y_range
        self.user_bias=Embedding(n_users, 1)
        self.movie_bias=Embedding(n_movies, 1)
    def forward(self, x):
        users=self.user_factors(x[:, 0])
        movies=self.movie_factors(x[:, 1])
        res=(users*movies).sum(dim=1, keepdim=True)
        res+=self.user_bias(x[:, 0])+self.movie_bias(x[:, 1])
        return sigmoid_range(res, *self.y_range)


In [13]:
model=DotProduct(n_users, n_movies, 50)
learn=Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.958137,0.954001,00:15
1,0.872298,0.875933,00:11
2,0.745986,0.838503,00:11
3,0.600054,0.821116,00:11
4,0.484811,0.821436,00:11


Create Embedding Class

In [14]:
class T(Module):
    def __init__(self): self.a=nn.Linear(1, 3, bias=False)
t=T()
L(t.parameters())
type(t.a.weight)

torch.nn.parameter.Parameter

In [15]:
def create_params(size):
    return nn.Parameter(torch.zeros(size).normal_(0, 0.01))
class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0, 5.5)):
        self.user_factors=create_params([n_users, n_factors])
        self.user_bias=create_params([n_users])
        self.movie_factors=create_params([n_movies, n_factors])
        self.movie_bias=create_params([n_movies])
        self.y_range=y_range
    def forward(self, x):
        users=self.user_factors[x[:, 0]]
        movies=self.movie_factors[x[:, 1]]
        res=(users*movies).sum(dim=1)
        res+=self.user_bias[x[:, 0]]+self.movie_bias[x[:, 1]]
        return sigmoid_range(res, *self.y_range)

In [16]:
model=DotProductBias(n_users, n_movies, 50)
learn=Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.935201,0.947195,00:11
1,0.847725,0.875162,00:11
2,0.734503,0.840965,00:12
3,0.579848,0.826163,00:11
4,0.482114,0.827025,00:12


In [17]:
movie_bias=learn.model.movie_bias.squeeze()
idxs=movie_bias.argsort()[-10:]
[dls.classes['title'][i] for i in idxs]

['Wrong Trousers, The (1993)',
 'Rear Window (1954)',
 'Good Will Hunting (1997)',
 'Casablanca (1942)',
 'Star Wars (1977)',
 'Silence of the Lambs, The (1991)',
 'Titanic (1997)',
 'Shawshank Redemption, The (1994)',
 "Schindler's List (1993)",
 'L.A. Confidential (1997)']

Using fastai.collab

In [18]:
learn=collab_learner(dls, n_factors=50, y_range=(0, 5.5))
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.972373,0.961815,00:11
1,0.855256,0.885220,00:11
2,0.755585,0.837792,00:12
3,0.576214,0.821853,00:11
4,0.501370,0.822821,00:11


In [19]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(944, 50)
  (i_weight): Embedding(1665, 50)
  (u_bias): Embedding(944, 1)
  (i_bias): Embedding(1665, 1)
)

In [40]:
movie_bias=learn.model.i_bias.weight.squeeze()
idxs=movie_bias.argsort(descending=True)[:5]
[dls.classes['title'][i] for i in idxs]
movie_factors=learn.model.i_weight.weight
idx=dls.classes['title'].o2i['Star Wars (1977)']
movie_factors[1]

tensor([-0.0420, -0.0990,  0.0405, -0.0638, -0.1353,  0.0460,  0.0444, -0.2186, -0.1368,  0.1050, -0.0899, -0.0659, -0.2621,  0.1128,  0.1426,  0.0815, -0.2505,  0.0013,  0.0349,  0.0614, -0.0808,
        -0.0187,  0.1205,  0.0765,  0.0197,  0.1981,  0.0159,  0.1123,  0.1830, -0.1101,  0.1371, -0.1850,  0.0914, -0.1623,  0.0517, -0.0509, -0.1292,  0.1471, -0.0633,  0.0558, -0.1043,  0.0449,
         0.0603, -0.1564, -0.1710,  0.0986, -0.1097, -0.1108, -0.0253, -0.1612], device='cuda:0', grad_fn=<SelectBackward0>)

In [41]:
distances=nn.CosineSimilarity(dim=1)(movie_factors, movie_factors[idx][None])
idx=distances.argsort(descending=True)[1]
dls.classes['title'][idx]

'Empire Strikes Back, The (1980)'

Deep Learning for Collaborative Filtering

In [43]:
embs=get_emb_sz(dls)
embs

[(944, 74), (1665, 102)]

In [44]:
class CollabNN(Module):
    def __init__(self, user_sz, item_sz, y_range=(0, 5.5), n_act=100):
        self.user_factors=Embedding(*user_sz)
        self.item_factors=Embedding(*item_sz)
        self.layers=nn.Sequential(
            nn.Linear(user_sz[1]+item_sz[1], n_act),
            nn.ReLU(),
            nn.Linear(n_act, 1)
        )
        self.y_range=y_range
    def forward(self, x):
        embs=self.user_factors(x[:,0]),self.item_factors(x[:, 1])
        x=self.layers(torch.cat(embs, dim=1))
        return sigmoid_range(x, *self.y_range)

In [48]:
model=CollabNN(*embs)
learn.fit_one_cycle(5, 5e-3, wd=0.1)


epoch,train_loss,valid_loss,time
0,0.270104,0.977890,00:11
1,0.426608,0.923448,00:11
2,0.404398,0.901758,00:10
3,0.303312,0.900438,00:10
4,0.211071,0.901595,00:11


Using collab learner from fastai with NN=on

In [50]:
learn=collab_learner(dls, use_nn=True, y_range=(0, 5.5), layers=[100, 50])
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.973819,0.979879,00:16
1,0.904278,0.925828,00:14
2,0.857492,0.890031,00:13
3,0.815672,0.866567,00:13
4,0.762642,0.868667,00:13
